In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sb


In [2]:
raw_data = pd.read_csv('D:\日常学习\毕业设计\数据\摩拜_上海_2016.8\mobike_shanghai_sample_updated.csv')
raw_data.head()


,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track
0,78387,158357,10080,2016/8/20 6:57,121.348,31.389,2016/8/20 7:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#1..."
1,891333,92776,6605,2016/8/29 19:09,121.508,31.279,2016/8/29 19:31,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#1..."
2,1106623,152045,8876,2016/8/13 16:17,121.383,31.254,2016/8/13 16:36,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#1..."
3,1389484,196259,10648,2016/8/23 21:34,121.484,31.320,2016/8/23 21:43,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#1..."
4,188537,78208,11735,2016/8/16 7:32,121.407,31.292,2016/8/16 7:41,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#1..."


In [3]:
raw_data.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   orderid           102361 non-null  int64  
 1   bikeid            102361 non-null  int64  
 2   userid            102361 non-null  int64  
 3   start_time        102361 non-null  object 
 4   start_location_x  102361 non-null  float64
 5   start_location_y  102361 non-null  float64
 6   end_time          102361 non-null  object 
 7   end_location_x    102361 non-null  float64
 8   end_location_y    102361 non-null  float64
 9   track             102361 non-null  object 
dtypes: float64(4), int64(3), object(3)
memory usage: 7.8+ MB


In [4]:
# 复制数据
m_bike = raw_data.copy()
m_bike['start_time'] = pd.to_datetime(m_bike['start_time']) # 时间转换成datetime
to_str = ['orderid', 'bikeid', 'userid']


In [5]:
m_bike[to_str] = m_bike[to_str].astype('str')

In [6]:
m_bike['end_time'] = pd.to_datetime(m_bike['end_time'])


In [7]:
m_bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   orderid           102361 non-null  object        
 1   bikeid            102361 non-null  object        
 2   userid            102361 non-null  object        
 3   start_time        102361 non-null  datetime64[ns]
 4   start_location_x  102361 non-null  float64       
 5   start_location_y  102361 non-null  float64       
 6   end_time          102361 non-null  datetime64[ns]
 7   end_location_x    102361 non-null  float64       
 8   end_location_y    102361 non-null  float64       
 9   track             102361 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(4)
memory usage: 7.8+ MB


In [8]:
# 新增骑行时间 duration，并将时间拆分成单位为min，命名为ttl_min

m_bike['duration'] = m_bike['end_time'] - m_bike['start_time']

In [9]:
# m_bike['dur_day'] = m_bike.duration.apply(lambda x: str(x).split(' ')[0])
m_bike['dur_day'] = m_bike.duration.apply(lambda x: str(x).split(' ')[0])

In [10]:
m_bike['dur_hour'] = m_bike.duration.apply(lambda x:str(x).split(' ')[-1][:2])


In [11]:
m_bike['dur_min'] = m_bike.duration.apply(lambda x:str(x).split(':')[-2])


In [12]:
m_bike['dur_sec'] = m_bike.duration.apply(lambda x:str(x).split(':')[-1])
to_int = ['dur_day', 'dur_hour', 'dur_min', 'dur_sec']
m_bike[to_int] = m_bike[to_int].astype('int')
m_bike['ttl_min'] = m_bike.dur_day*24*60+m_bike.dur_hour*60+m_bike.dur_min+m_bike.dur_sec/60

In [13]:
# day_id 和day_type ,每条记录为星期几，并根据工作日和周末进行分类
m_bike['day_id'] = m_bike.start_time.apply(lambda x: x.isoweekday())
m_bike['day_type'] = m_bike.day_id.apply(lambda x: 'weekends' if x == 7 or x == 6 else 'weekdays')

In [14]:
# 新增hour_id 和hour_type，获取每条记录的开始时间点，并根据早晚高峰和平和时间段分类

m_bike['hour_id'] = m_bike.start_time.apply(lambda x:x.utctimetuple().tm_hour)


In [15]:
#  按照小时的时间段进行分类

m_bike['hour_type'] = m_bike.hour_id.apply(lambda x:'rush_hour' if (x <= 9 and x >=7) or (x >= 17 and x <= 20) else 'non_rush_hour')

In [16]:
# 新增地理位置信息相关的列：
# distance 和 distocenter 距离上海市中心的距离（km）
from math import cos, sin , sqrt, asin, radians


def geo_distance(lng1, lat1, lng2, lat2):
    """
    # 根据经纬度计算亮点之间的直线距离
    :param lng1: 位置1经度
    :param lat1: 维度1
    :param lng2: 经度2
    :param lat2: 纬度2

    :return: 二者之间直线距离
    """
    lng1_r, lat1_r, lng2_r, lat2_r = map(radians, [lng1, lat1, lng1, lat2]) # map函数，转换成弧度
    d_lng = lng1_r - lng2_r # 经度差值（弧度）
    d_lat = lat1_r - lat2_r # 纬度差值
    dis = sin(d_lat/2)**2 + cos(lat1_r)*cos(lat2_r)*sin(d_lng/2)**2
    distance = 2 * asin(sqrt(dis))*6738.137*1000
    distance = round(distance/1000,3) # 保留三位小数 ，单位km
    return distance

def get_dis(item):
    item['distance'] = geo_distance(item['start_location_x'], item['start_location_y'],
                                   item['end_location_x'], item['end_location_y'])    # 计算骑行始末点经纬度的直线距离
    # 国际饭店一般被认为是上海地理中心坐标点，计算骑行起始点经纬度和国际饭店经纬度的直线距离
    item['dis_to_center'] = geo_distance(item['start_location_x'], item['start_location_y'], 121.471632, 31.233705)
    return item

In [17]:
m_bike['distance'] = np.nan
m_bike['dis_to_center'] = np.nan
m_bike = m_bike.apply(get_dis, axis=1)

In [19]:
# 按照地点距离市中心的距离 进行划分 ring_state

def get_ring_state(item):
    if item['dis_to_center'] <= 10:
        item['ring_stage'] = 'inside inner ring' # 外环
    elif item['dis_to_center'] <= 15:
        item['ring_stage'] = 'inside middle ring'    # 中环距国际饭店最远距离约为15km
    elif item['dis_to_center'] <= 18:
        item['ring_stage'] = 'inside outer ring'    # 外环距国际饭店最远距离约为18km
    elif item['dis_to_center'] > 18:
        item['ring_stage'] = 'outside outer ring'
    return item

m_bike['ring_stage'] = np.nan
m_bike = m_bike.apply(get_ring_state, axis=1)

In [20]:
m_bike = m_bike.drop(labels='ring_state',axis=1)

In [21]:
import math
m_bike['cost'] = m_bike.ttl_min.apply(lambda x: math.ceil(x/30))    # 参照2016年摩拜收费标准，按每30分钟收取1元进行cost列的粗略计算


In [22]:
m_bike_sub = m_bike.copy()
m_bike_sub.userid = m_bike_sub.userid.astype('int')

In [23]:
m_bike_sub = m_bike_sub.sort_values('userid')

In [ ]:
# 计算RFM值
